In [1]:
import re
import time
import requests
import pandas as pd
from io import StringIO
from bs4 import BeautifulSoup
from fuzzymatcher import link_table, fuzzy_left_join

In [45]:

import fresh_data.get_datasets
import importlib
importlib.reload(fresh_data.get_datasets)
from fresh_data.get_datasets import *

In [46]:
fec = load_FEC_data("FEC/")
fec

,year_range,state_name,district_code,representative,party,running_as,receipts,contributions_from_individuals,contributions_from_pacs,contributions_and_loans_from_candidate,disbursements,cash_on_hand,debts,congress
0,1989-1991,Alabama,1,"CALLAHAN, HERBERT L",Republican Party,Incumbent,318680.00,129153.00,169350.00,0.00,183910.00,236511.00,0.0,101
1,1989-1991,Alabama,2,"DICKINSON, WILLIAM LOUIS",Republican Party,Incumbent,425127.00,171618.00,218921.00,0.00,596096.00,250325.00,0.0,101
2,1989-1991,Alabama,2,"BAGGIANO, FAYE S",Democratic Party,Challenger,169109.00,38099.00,91752.00,20993.00,163663.00,5445.00,0.0,101
3,1989-1991,Alabama,3,"ADAMS, CHARLES H",Democratic Party,Open,277954.00,122590.00,19400.00,0.00,276975.00,978.00,55000.0,101
4,1989-1991,Alabama,3,"DIAL, GERALD OSCAR",Democratic Party,Open,111460.00,22420.00,14750.00,74290.00,111459.00,0.00,37894.0,101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24782,2019-2021,Wisconsin,8,"STUCK, AMANDA",Democratic Party,Challenger,424558.24,390832.60,30183.44,0.00,407495.73,2164.80,0.0,116
24783,2019-2021,Wyoming,1,"CHENEY, ELIZABETH MRS.",Republican Party,Incumbent,3010251.47,1169995.46,1292490.00,0.00,3066534.91,153567.15,70000.0,116
24784,2019-2021,Wyoming,1,"BEACH, CARL",Democratic Party,Challenger,8591.85,8591.85,0.00,0.00,8591.85,0.00,0.0,116
24785,2019-2021,Wyoming,1,"GREY BULL, LYNNETTE",Democratic Party,Challenger,134648.22,130197.32,2800.00,0.00,132285.65,2362.57,0.0,116


In [47]:
fec_116 = fec[fec["congress"]==116]
fec_116["state_name"].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
       'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype=object)

In [26]:
state_district_mask = (fec["state_name"] == "New York") & (fec["district_code"] == 12)
fec[state_district_mask]

,year_range,state_name,district_code,representative,party,running_as,receipts,contributions_from_individuals,contributions_from_pacs,contributions_and_loans_from_candidate,disbursements,cash_on_hand,debts,congress
696,1989-1991,New York,12,"OWENS, MAJOR ROBERT",Democratic Party,Incumbent,168086.00,42220.00,115025.00,0.00,175998.00,6451.00,24861.00,101
2197,1991-1993,New York,12,"SOLARZ, STEPHEN J",Democratic Party,Incumbent,1526662.00,1214735.00,55800.00,0.00,3235457.00,150808.00,0.00,102
2198,1991-1993,New York,12,"COLON, ELIZABETH",Democratic Party,Challenger,307335.00,147929.00,1750.00,157200.00,270670.00,1713.00,123200.00,102
2199,1991-1993,New York,12,"DIAZ, ANGEL",Republican Party,Challenger,5300.00,0.00,0.00,4500.00,5390.00,0.00,1968.00,102
2200,1991-1993,New York,12,"FRANCO, RUBEN",Liberal Party,Challenger,88971.00,17840.00,0.00,44884.00,85840.00,3130.00,76623.00,102
2201,1991-1993,New York,12,"LUI, ALFRED S",Independent,Challenger,13268.00,11140.00,0.00,2128.00,13267.00,0.00,0.00,102
2202,1991-1993,New York,12,"VELAZQUEZ, NYDIA M",Democratic Party,Challenger,482053.00,208153.00,153900.00,20000.00,447787.00,28406.00,273628.00,102
3681,1993-1995,New York,12,"VELAZQUEZ, NYDIA M",Democratic Party,Incumbent,624095.00,306225.00,217470.00,103500.00,605787.00,45913.00,38494.00,103
3682,1993-1995,New York,12,"VELAZQUEZ, PEDRO L",Democratic Party,Challenger,11574.00,3570.00,2000.00,2500.00,11360.00,215.00,9946.00,103
5148,1995-1997,New York,12,"VELAZQUEZ, NYDIA M",Democratic Party,Incumbent,295001.00,185261.00,108239.00,0.00,238014.00,104065.00,12312.00,104


In [27]:
fec[fec["congress"]==116]

,year_range,state_name,district_code,representative,party,running_as,receipts,contributions_from_individuals,contributions_from_pacs,contributions_and_loans_from_candidate,disbursements,cash_on_hand,debts,congress
22705,2019-2021,Alabama,1,"AVERHART, JAMES",Democratic Party,Open,80094.95,50849.95,0.00,29245.00,78973.24,1121.71,29245.00,116
22706,2019-2021,Alabama,1,"CARL, JERRY LEE, JR",Republican Party,Open,2344516.53,1044195.95,387000.00,760555.50,2232543.94,111972.59,469225.79,116
22707,2019-2021,Alabama,1,"CASTORANI, JOHN",Republican Party,Open,26809.68,11650.00,0.00,15159.68,26809.68,0.00,6559.68,116
22708,2019-2021,Alabama,1,"COLLINS, FREDERICK G. RICK'",Democratic Party,Open,62935.42,5917.12,0.00,56500.00,60825.97,2109.45,0.00,116
22709,2019-2021,Alabama,1,"GARDNER, KIANI A",Democratic Party,Open,118661.85,92896.88,19000.00,6764.97,118661.85,0.00,5861.09,116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24782,2019-2021,Wisconsin,8,"STUCK, AMANDA",Democratic Party,Challenger,424558.24,390832.60,30183.44,0.00,407495.73,2164.80,0.00,116
24783,2019-2021,Wyoming,1,"CHENEY, ELIZABETH MRS.",Republican Party,Incumbent,3010251.47,1169995.46,1292490.00,0.00,3066534.91,153567.15,70000.00,116
24784,2019-2021,Wyoming,1,"BEACH, CARL",Democratic Party,Challenger,8591.85,8591.85,0.00,0.00,8591.85,0.00,0.00,116
24785,2019-2021,Wyoming,1,"GREY BULL, LYNNETTE",Democratic Party,Challenger,134648.22,130197.32,2800.00,0.00,132285.65,2362.57,0.00,116


In [28]:
fec.year_range.unique()

array(['1989-1991', '1991-1993', '1993-1995', '1995-1997', '1997-1999',
       '1999-2001', '2001-2003', '2003-2005', '2005-2007', '2007-2009',
       '2009-2011', '2011-2013', '2013-2015', '2015-2017', '2017-2019',
       '2019-2021'], dtype=object)